In [1]:
import pandas as pd
import seaborn as sns
import fbprophet 
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [2]:
# total population by county (adjusted to 2010 controls)
county_2010=pd.read_csv('../../data/NHGIS/nhgis0001_csv/nhgis0001_ts_geog2010_county.csv')

# total population by place
pop_by_place=pd.read_csv('../../data/NHGIS/nhgis0002_csv/nhgis0002_ts_nominal_place.csv',encoding='ISO-8859-1')
# nyc=pop_by_place.iloc[[18672]]
# nyc


# total population by county (1790 to 2010)
by_state_county=pd.read_csv('../../data/NHGIS/nhgis0003_csv/nhgis0003_ts_nominal_county.csv',encoding='ISO-8859-1')

In [ ]:
# explore by_state_county 
# expect total population by county 1790-2010
by_state_county.info()

In [ ]:
by_state_county.describe()

- ***notes***:
    - looks to be in line with other data
- ***the txt***:

    Context Fields 
        GISJOIN:     GIS Join Match Code
        STATE:       NHGIS Integrated State Name
        STATEFP:     FIPS State Code
        STATENH:     NHGIS Integrated State Code
        COUNTY:      NHGIS Integrated County Name
        COUNTYFP:    FIPS County Code
        COUNTYNH:    NHGIS Integrated County Code
 
    Table 1: (A00) Total Population
        Time series AA: Persons: Total
            A00AA1790:   1790: Persons: Total
            A00AA1800:   1800: Persons: Total
            A00AA1810:   1810: Persons: Total
            A00AA1820:   1820: Persons: Total
            A00AA1830:   1830: Persons: Total
            A00AA1840:   1840: Persons: Total
            A00AA1850:   1850: Persons: Total
            A00AA1860:   1860: Persons: Total
            A00AA1870:   1870: Persons: Total
            A00AA1880:   1880: Persons: Total
            A00AA1890:   1890: Persons: Total
            A00AA1900:   1900: Persons: Total
            A00AA1910:   1910: Persons: Total
            A00AA1920:   1920: Persons: Total
            A00AA1930:   1930: Persons: Total
            A00AA1940:   1940: Persons: Total
            A00AA1950:   1950: Persons: Total
            A00AA1960:   1960: Persons: Total
            A00AA1970:   1970: Persons: Total
            A00AA1980:   1980: Persons: Total
            A00AA1990:   1990: Persons: Total
            A00AA2000:   2000: Persons: Total
            A00AA2010:   2010: Persons: Total

In [ ]:
by_state_county.head(3)

In [ ]:
# GISJOIN:     GIS Join Match Code
print(len(by_state_county.GISJOIN.unique()),len(by_state_county.GISJOIN))

#   STATE:       NHGIS Integrated State Name
print(len(by_state_county.STATE.unique()),len(by_state_county.STATE))

#   STATEFP:     FIPS State Code
print(len(by_state_county.STATEFP.unique()),len(by_state_county.STATEFP))

#   STATENH:     NHGIS Integrated State Code
print(len(by_state_county.STATENH.unique()),len(by_state_county.STATENH))

#   COUNTY:      NHGIS Integrated County Name
print(len(by_state_county.COUNTY.unique()),len(by_state_county.COUNTY))

#   COUNTYFP:    FIPS County Code
print(len(by_state_county.COUNTYFP.unique()),len(by_state_county.COUNTYFP))

#   COUNTYNH:    NHGIS Integrated County Code
print(len(by_state_county.COUNTYNH.unique()),len(by_state_county.COUNTYNH))

- ***action***:
    - keep GISJOIN, drop else

In [5]:
# total population by county (1790 to 2010)
by_state_county=pd.read_csv('../../data/NHGIS/nhgis0003_csv/nhgis0003_ts_nominal_county.csv',encoding='ISO-8859-1')

"""
- generate DataFrame of population:
    > from 1790 to 2010
    > by unique county (use GIS Join as Id)
- drop 
    > counties with less than 2 measurements
        > can only predict counties which have been measured 2+ times 
- extract list of counties
    > each as a DataFrame ready for prediction 
    > column0='ds' , column1='y'
"""

# df by GIS Join Match Code with measurements by decade (4050 rows × 24 columns)
unique_counties = by_state_county.copy()[['GISJOIN','A00AA1790','A00AA1800','A00AA1810','A00AA1820','A00AA1830','A00AA1840','A00AA1850','A00AA1860',
               'A00AA1870','A00AA1880','A00AA1890','A00AA1900','A00AA1910','A00AA1920','A00AA1930','A00AA1940','A00AA1950',
                'A00AA1960','A00AA1970','A00AA1980','A00AA1990','A00AA2000','A00AA2010']]

# drop NaN rows @ thresh = 3 due to GISJOIN being non-NaN (3408 rows × 24 columns ; 642 non-predictable counties (dropped)) 
measureable_unique_counties = unique_counties.dropna(axis=0,thresh=3)
# convert NaN values to 0 (note: there are 270 'dead' counties ('A00AA2010' == 0))
measureable_unique_counties = measureable_unique_counties.fillna(0)

# generate list of remaining GISJOIN codes 
codes_of_measureable_unique_counties = [code for code in measureable_unique_counties.GISJOIN]
# drop GISJOIN column
measureable_unique_counties=measureable_unique_counties.drop('GISJOIN',axis=1)

# list of str column names as years (for conversion to datetime)
year_only_columns = [i[5:] for i in measureable_unique_counties.columns]
# convert year_only_columns to DatetimeIndex of Timestamps
dt_columns = pd.to_datetime(arg=year_only_columns)
# convert dt_columns into dataframe 
datetime_df = pd.DataFrame(dt_columns).T
# w/ columns, so concatable with measureable_unique_counties
datetime_df.columns = measureable_unique_counties.columns

# generate list of remaining counties (each as pd.Series)
dfs_of_measureable_unique_counties = [measureable_unique_counties.iloc[county] for county in range(len(measureable_unique_counties))]

# add datetime_df to each dataframe as first row
prophet_counties = [pd.concat((datetime_df,pd.DataFrame(county).T),axis=0) for county in dfs_of_measureable_unique_counties]
# then transpose to 2 rows x 23 columns 
prophet_almost_ready_counties = [county.T for county in prophet_counties]

# set collection of prophets 
prophet_by_county = []

# run prophet model on each county 
for county in range(len(prophet_almost_ready_counties)):
    # make the prophet model
    county_prophet = fbprophet.Prophet(changepoint_prior_scale=0.15)
    # identify county 
    a = prophet_almost_ready_counties[county]
    # rename county df's columns to agree with prophet formatting
    a.columns = ['ds','y']
    # fit county on prophet model 
    b = county_prophet.fit(a)
    # make a future dataframe for 20 years
    county_forecast = county_prophet.make_future_dataframe( periods=1*20, freq='Y' )
    # establish predictions
    county_forecast = county_prophet.predict(county_forecast)
    # add to collection 
    prophet_by_county.append(county_forecast)

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 17.0.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 17.0.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 17.0.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. 

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 17.0.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 17.0.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 17.0.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. 

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 17.0.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 17.0.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 17.0.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. 

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 17.0.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 17.0.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 17.0.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. 

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 17.0.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 17.0.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 17.0.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. 

...

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 17.0.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 17.0.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 17.0.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. 

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 17.0.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 17.0.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 17.0.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. 

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 17.0.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 17.0.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 17.0.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. 

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 17.0.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 17.0.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 17.0.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. 

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 17.0.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 17.0.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 17.0.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. 

In [6]:
len(prophet_by_county)

3408

In [7]:
type(prophet_by_county[0])

pandas.core.frame.DataFrame

In [9]:
prophet_by_county[0]

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,1790-01-01,-409.333514,-4177.525298,9144.280153,-409.333514,-409.333514,2754.605377,2754.605377,2754.605377,2754.605377,2754.605377,2754.605377,0.0,0.0,0.0,2345.271863
1,1800-01-01,1559.493841,-4184.867430,9282.780921,1559.493841,1559.493841,897.613578,897.613578,897.613578,897.613578,897.613578,897.613578,0.0,0.0,0.0,2457.107419
2,1810-01-01,3528.321167,-4056.230222,8964.183638,3528.321167,3528.321167,-972.006668,-972.006668,-972.006668,-972.006668,-972.006668,-972.006668,0.0,0.0,0.0,2556.314499
3,1820-01-01,5497.148500,-3897.306989,9182.429946,5497.148500,5497.148500,-2845.180822,-2845.180822,-2845.180822,-2845.180822,-2845.180822,-2845.180822,0.0,0.0,0.0,2651.967678
4,1830-01-01,7466.514939,-263.394429,13553.343993,7466.514939,7466.514939,-972.006668,-972.006668,-972.006668,-972.006668,-972.006668,-972.006668,0.0,0.0,0.0,6494.508272
5,1840-01-01,9435.342207,-72.486304,13580.094789,9435.342207,9435.342207,-2845.180822,-2845.180822,-2845.180822,-2845.180822,-2845.180822,-2845.180822,0.0,0.0,0.0,6590.161385
6,1850-01-01,11404.708583,3625.827040,17034.551427,11404.708583,11404.708583,-972.006668,-972.006668,-972.006668,-972.006668,-972.006668,-972.006668,0.0,0.0,0.0,10432.701916
7,1860-01-01,13373.535847,3959.677023,17737.187848,13373.535847,13373.535847,-2845.180822,-2845.180822,-2845.180822,-2845.180822,-2845.180822,-2845.180822,0.0,0.0,0.0,10528.355025
8,1870-01-01,15342.902222,7481.831465,21309.391040,15342.902222,15342.902222,-972.006668,-972.006668,-972.006668,-972.006668,-972.006668,-972.006668,0.0,0.0,0.0,14370.895555
9,1880-01-01,17311.729486,7940.626097,21275.670516,17311.729486,17311.729486,-2845.180822,-2845.180822,-2845.180822,-2845.180822,-2845.180822,-2845.180822,0.0,0.0,0.0,14466.548664


In [ ]:
"""
generate list of counties by name as string type;
format: 'COUNTY, STATE'
e.g. 'Autauga County, Alabama'
"""
state_and_county_ = by_state_county.copy()[['COUNTY','STATE']]
state_county_names_list = [state_and_county_.loc[i][0] + ', ' + state_and_county_.loc[i][1] for i in range(len(state_and_county_))]
state_county_names_list

In [ ]:
4050-3408